In [1]:

from utils.config_loader import app_config
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

from pymilvus import connections, list_collections
import textwrap

from llama_index.core.vector_stores import MetadataFilters,  ExactMatchFilter


In [2]:
import logging
logging.basicConfig(level=logging.ERROR)

### Example 1: Using LlamaIndex to query the Milvus vector store

In [3]:
# Step 0 . Connect to Milvus
try:
    # 1. Connect to Milvus
    milvus_store = MilvusVectorStore(
        uri=app_config.MILVUS_URL,
        dim=app_config.MILVUS_DIM,        
        overwrite=False,
        collection_name="_emails_attachments"
    )
    print("Connected to Milvus successfully.")
except Exception as e:
    print(f"Failed to connect to Milvus: {e}")

2025-04-17 04:49:52,364 [DEBUG][_create_connection]: Created new connection using: bb60ca7411c84b788f074c9afd5f5ad1 (async_milvus_client.py:600)


Connected to Milvus successfully.


In [4]:
# Step 0: Optional. To see the contents of a search result. 

print(milvus_store.client.search(
    collection_name="_emails_attachments", 
    data=[[0.1] * app_config.MILVUS_DIM],
    limit=1,
    output_fields=["*"],
))

data: ['[{\'id\': \'ee9b0408-a231-4f92-ba25-e33ec8e0d8a6\', \'distance\': -0.10918539762496948, \'entity\': {\'id\': \'ee9b0408-a231-4f92-ba25-e33ec8e0d8a6\', \'embedding\': [-0.006668765097856522, -0.025433717295527458, 0.00682738097384572, -0.02266138792037964, -0.02274414338171482, -0.0014223710168153048, -0.02142004482448101, -0.02206830121576786, -0.0021102814935147762, -0.0067928992211818695, 0.002563715912401676, 0.03122664801776409, -0.010482443496584892, 0.008082515560090542, 0.002746469108387828, -1.0506149692446343e-06, 0.012220322154462337, 0.0006978238234296441, -0.02351653389632702, -0.014675420708954334, -0.013282359577715397, -0.0028102602809667587, -0.01932355761528015, -0.0053446670062839985, -0.007537704426795244, -0.0027206079103052616, 0.032247304916381836, -0.026330243796110153, 0.009061796590685844, 0.011206559836864471, 0.0027292282320559025, 0.009413509629666805, -0.02805432863533497, 0.014868518337607384, -0.007061856798827648, -0.0046481359750032425, -0.00061

In [5]:
## Step 1: Prepare global settings

# Access appplication environment variables
model = app_config.LLAMAINDEX_MODEL
embed_model= app_config.LLAMAINDEX_EMBEDDING

# define global Setting that sets the LLM and ebemdding model.
# There is no need to pass in any llm or embedding model to the LlamaIndexTool query engine.
# The LlamaIndexTool will use the global Settings to get the llm and embedding model.
Settings.llm = OpenAI(temperature=0.1, model=model)
Settings.embed_model = OpenAIEmbedding(model=embed_model)

In [6]:
# Step 2: Load index from existing Milvus vector store
index = VectorStoreIndex.from_vector_store(
    vector_store=milvus_store
)

In [7]:
# Step 3. Create the query and chat engine using the LLM
query_engine = index.as_query_engine()  # no parameter needed, taken care off by the global Settings
chat_engine = index.as_chat_engine()


queries = [
    "what did Juliet Iris Lee reqeusted for?",
    "what are items shipped?",
    "where is the destination terminal of the shipment?  Provide the destination address.",
    "find items names and qauntity in move order 6009981",
    "when is the requested truck in date and time?",
    "identify the vessel name and voyage number for the job",
    ## add more queries here ##
]

In [9]:

# Step 4A. Run your query
for q in queries:
    response = query_engine.query(q)
    print(textwrap.fill(response.response, width=80))
    print()

There is no indication that Juliet Iris Lee made any specific request. The
message primarily informs her about a vessel delay.

The specific items shipped are not detailed in the provided information.

The destination terminal of the shipment is Jakarta. However, the specific
destination address is not provided.

The items and their quantities in move order 6009981 are as follows:  1. CLOVER
(KF) WHEAT FLOUR @ 25 KG NETT - 72 bags 2. IVORY (F) WHEAT FLOUR @ 25 KG NETT -
72 bags 3. TOWER (F) WHEAT FLOUR @ 20 KG - 80 bags 4. FAIRY (F) WHEAT FLOUR @ 25
KG NETT - 646 bags

The requested truck-in date is 02 October, which falls on a Wednesday. There is
no specific time mentioned.

The vessel name and voyage number are not specified in the provided information.



In [8]:
# Step 4B. Run your  chat engine

for q in queries:
    response = chat_engine.chat(q)
    print(textwrap.fill(response.response, width=80))
    print()



Juliet Iris Lee was informed about a vessel delay until October 15, 2024.

The context does not provide specific details about the items shipped.

The destination terminal of the shipment and its address are not specified in
the provided information.

The items and quantities in move order 6009981 are:  1. CLOVER (KF) WHEAT FLOUR
@ 25 KG NETT - 1800 KG (72 BAGS) 2. IVORY (F) WHEAT FLOUR @ 25 KG NETT - 1800 KG
(72 BAGS) 3. TOWER (F) WHEAT FLOUR @ 20 KG - 1600 KG (80 BAGS) 4. FAIRY (F)
WHEAT FLOUR @ 25 KG NETT - 16150 KG (646 BAGS)

The requested truck-in date is October 2nd, which falls on a Wednesday. The
specific time is not mentioned.

The vessel name and voyage number for the job are not provided in the available
information.



### Example 2: Using LlamaIndex to query the Milvus vector store using Meta Data Filters

In [11]:

query_engine_with_filters = index.as_query_engine(
        include_metadata=True,
        vector_store_query_mode="default",
        filters=MetadataFilters(
        filters=[
           # ExactMatchFilter(key="threadID", value="194d494969f09f62"),
        ]
        )
)

for q in queries:
    response = query_engine_with_filters.query(q)
    print(textwrap.fill(response.response, width=80))
    print()

The context does not provide specific details about any request made by Juliet
Iris Lee.

The items shipped are various types of wheat flour, including CLOVER (KF) WHEAT
FLOUR, IVORY (F) WHEAT FLOUR, TOWER (F) WHEAT FLOUR, and FAIRY (F) WHEAT FLOUR.

The destination terminal of the shipment is Jakarta. However, the specific
address for the destination terminal is not provided.

The items and their quantities in move order 6009981 are as follows:  1. CLOVER
(KF) WHEAT FLOUR @ 25 KG NETT - 72 bags 2. IVORY (F) WHEAT FLOUR @ 25 KG NETT -
72 bags 3. TOWER (F) WHEAT FLOUR @ 20 KG - 80 bags 4. FAIRY (F) WHEAT FLOUR @ 25
KG NETT - 646 bags

The requested truck-in date is 02 October, and no specific time is mentioned.

